In [ ]:
# | default_exp _cli

In [ ]:
# | export

import importlib
import sys
import asyncio
from pathlib import Path
from typing import *
import signal
from os import getpid
import time
import anyio
import threading

import typer
from fastapi import FastAPI

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger, supress_timestamps

In [ ]:
import os
from contextlib import contextmanager
from tempfile import TemporaryDirectory

import nbformat
from nbconvert import PythonExporter
from typer.testing import CliRunner

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
@contextmanager
def cwd(path: Union[str, Path]) -> None:
    org_cwd = os.getcwd()
    try:
        os.chdir(path)
        yield
    finally:
        os.chdir(org_cwd)


with cwd("/tmp"):
    assert os.getcwd() == "/tmp"
assert os.getcwd() != "/tmp"
os.getcwd()

'/work/fastkafka/nbs'

In [ ]:
def generate_app_src(out_path: Union[Path, str]) -> None:
    path = Path("099_Test_Service.ipynb")
    if not path.exists():
        path = Path("..") / "099_Test_Service.ipynb"
    if not path.exists():
        raise ValueError(f"Path '{path.resolve()}' does not exists.")

    with open(path, "r") as f:
        notebook = nbformat.reads(f.read(), nbformat.NO_CONVERT)
        exporter = PythonExporter()
        source, _ = exporter.from_notebook_node(notebook)

    with open(out_path, "w") as f:
        f.write(source)

In [ ]:
with TemporaryDirectory() as d:
    generate_app_src((Path(d) / "main.py"))
    !ls -al {d}
    !cat {d}/main.py | grep @app

total 11
drwx------  2 tvrtko tvrtko     3 Jan 27 16:22 .
drwxrwxrwt 11 root   root      14 Jan 27 16:22 ..
-rw-rw-r--  1 tvrtko tvrtko 10831 Jan 27 16:22 main.py


In [ ]:
# | export


class ImportFromStringError(Exception):
    pass


def _import_from_string(import_str: str) -> Any:
    """Imports library from string

    Note:
        copied from https://github.com/encode/uvicorn/blob/master/uvicorn/importer.py

    Args:
        import_str: input string in form 'main:app'

    """
    sys.path.append(".")

    if not isinstance(import_str, str):
        return import_str

    module_str, _, attrs_str = import_str.partition(":")
    if not module_str or not attrs_str:
        message = (
            'Import string "{import_str}" must be in format "<module>:<attribute>".'
        )
        typer.secho(f"{message}", err=True, fg=typer.colors.RED)
        raise ImportFromStringError(message.format(import_str=import_str))

    try:
        # nosemgrep: python.lang.security.audit.non-literal-import.non-literal-import
        module = importlib.import_module(module_str)
    except ImportError as exc:
        if exc.name != module_str:
            raise exc from None
        message = 'Could not import module "{module_str}".'
        raise ImportFromStringError(message.format(module_str=module_str))

    instance = module
    try:
        for attr_str in attrs_str.split("."):
            instance = getattr(instance, attr_str)
    except AttributeError:
        message = 'Attribute "{attrs_str}" not found in module "{module_str}".'
        raise ImportFromStringError(
            message.format(attrs_str=attrs_str, module_str=module_str)
        )

    return instance

In [ ]:
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with cwd(d):
#         rest_app = _import_from_string(f"{src_path.stem}:rest_app")
        kafka_app = _import_from_string(f"{src_path.stem}:kafka_app")
#         assert isinstance(rest_app, FastAPI)
        assert isinstance(kafka_app, FastKafka)

[INFO] main: check


In [ ]:
runner = CliRunner()

In [ ]:
class KafkaWorkersHandler:
    def __init__(
        self,
        *,
        app: FastKafka,
        num_workers: int = 1,
    ) -> None:
        self.app = app
        self.num_workers = num_workers
        self.should_exit = False

    def run(self) -> None:
        return asyncio.run(self.serve())

    async def serve(self) -> None:
        process_id = getpid()

        self.install_signal_handlers()

        message = f"Starting workers in process: {process_id}"
        logger.info(message)

        
        self._bg_task_group_generator = anyio.create_task_group()
        self._bg_tasks_group = await self._bg_task_group_generator.__aenter__()
        self._bg_tasks_group.start_soon(self.app.serve)
        await self.main_loop()
        self._bg_tasks_group.cancel_scope.cancel()  # type: ignore
        await self._bg_task_group_generator.__aexit__(None, None, None)  # type: ignore

        message = f"Stopped workers in process: {process_id}"
        logger.info(message)

    def install_signal_handlers(self) -> None:
        if threading.current_thread() is not threading.main_thread():
            # Signals can only be listened to from the main thread.
            return

        loop = asyncio.get_event_loop()

        HANDLED_SIGNALS = (
            signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
            signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
        )

        for sig in HANDLED_SIGNALS:
            loop.add_signal_handler(sig, self.handle_exit, sig)

    def handle_exit(self, sig: int) -> None:
        self.should_exit = True

    async def main_loop(self) -> None:
        while not self.should_exit:
            await asyncio.sleep(0.1)

In [ ]:
# | export

_app = typer.Typer(help="")


@_app.command(
    help="Runs Fast Kafka API application",
)
def run(
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        worker_handler = KafkaWorkersHandler(app=application, num_workers=1)
        worker_handler.run()
    except Exception as e:
        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)


@_app.command(
    help="Creates documentation for a Fast Kafka API application ",
)
def generate_docs(
    root_path: str = typer.Option(
        ".", help="root path under which documentation will be create"
    ),
    app: str = typer.Argument(
        ...,
        help="input in the form of 'path:app', where **path** is the path to a python file and **app** is an object of type **FastKafka**.",
    ),
) -> None:
    try:
        application = _import_from_string(app)
        application.skip_docs = False
        application.create_docs()
    except Exception as e:

        typer.secho(f"Unexpected internal error: {e}", err=True, fg=typer.colors.RED)
        raise typer.Exit(1)

In [ ]:
result = runner.invoke(_app, ["--help"])

                                                                                                                   
 Usage: root [OPTIONS] COMMAND [ARGS]...                                                                           
                                                                                                                   

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --install-completion          Install completion for the current shell.                                         │
│ --show-completion             Show completion for the current shell, to copy it or customize the installation.  │
│ --help                        Show this message and exit.                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Commands ──────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ generate-docs           Creates documentation for a Fast Kafka API application                                  │
│ run                     Runs Fast Kafka API application                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["generate-docs", "--help"])

                                                                                                                   
 Usage: root generate-docs [OPTIONS] APP                                                                           
                                                                                                                   

Creates documentation for a Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --root-path        TEXT  root path under which documentation will be create [default: .]                        │
│ --help                   Show this message and exit.                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = runner.invoke(_app, ["run", "--help"])

                                                                                                                   
 Usage: root run [OPTIONS] APP                                                                                     
                                                                                                                   

Runs Fast Kafka API application

╭─ Arguments ─────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ *    app      TEXT  input in the form of 'path:app', where **path** is the path to a python file and **app** is │
│                     an object of type **FastKafka**.                                                            │
│                     [default: None]                                                                             │
│                     [required]                                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Options ───────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ --help          Show this message and exit.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# | notest
with TemporaryDirectory() as d:
    src_path = Path(d) / "main.py"
    generate_app_src(src_path)
    with cwd(d):
        import_str = f"{src_path.stem}:kafka_app"

        result = runner.invoke(_app, ["run", import_str])
        typer.echo(result.output)
        assert result.exit_code == 0

[INFO] __main__: Starting workers in process: 2667
[INFO] fastkafka.application: Started server process 2667
[INFO] fastkafka._components.asyncapi: Old async specifications at '/tmp/tmplryrrpsl/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fastkafka._components.asyncapi: New async specifications generated at: '/tmp/tmplryrrpsl/asyncapi/spec/asyncapi.yml'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'tvrtko-fastkafka-kafka-1:9092'}'
[INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_cons

CancelledError: 